In [19]:
import os
import numpy as np
import faiss
import openai
from langdetect import detect
from dotenv import load_dotenv

load_dotenv()
# 환경 변수에서 API 키 가져오기
api_key = os.getenv('OPENAI_API_KEY')

if api_key is None:
    raise ValueError("OPENAI_API_KEY 환경 변수가 설정되지 않았습니다.")
openai.api_key = api_key

# 현재 디렉토리 설정
current_dir = os.getcwd()
vector_db_dir = os.path.join(current_dir, 'VectorDB')

# FAISS 인덱스 로드
index = faiss.read_index(os.path.join(vector_db_dir, 'vector_db.index'))
index_dimension = index.d  # 인덱스의 차원 확인

# 파일 경로 로드
with open(os.path.join(vector_db_dir, 'file_paths.txt'), 'r', encoding='utf-8') as f:
    file_paths = [line.strip() for line in f]

# 모든 문서 로드
documents = []
for path in file_paths:
    with open(path, 'r', encoding='utf-8') as file:
        documents.append(file.read())

# 임베딩 생성 함수
def get_embedding(text):
    response = openai.Embedding.create(
        input=text,
        engine="text-embedding-ada-002"
    )
    return np.array(response['data'][0]['embedding'], dtype=np.float32)

def search(query, top_k=5):
    # 쿼리 벡터화
    query_embedding = get_embedding(query).reshape(1, -1)
    
    # 차원 일치 여부 확인
    if query_embedding.shape[1] != index_dimension:
        raise ValueError(f"Query embedding dimension {query_embedding.shape[1]} does not match index dimension {index_dimension}")

    # FAISS 인덱스에서 유사한 문서 검색
    D, I = index.search(query_embedding, top_k)
    
    results = [(file_paths[i], documents[i]) for i in I[0]]
    return results

def generate_answer(query, top_k=1):
    # 관련 문서 검색
    relevant_docs = search(query, top_k)
    
    # 관련 문서들을 하나의 문자열로 결합
    context = "\n\n".join(doc for _, doc in relevant_docs)
    references = "\n".join(path for path, _ in relevant_docs)
    
    language = detect(query)

    if language == 'ko':
        system_message = "You are a helpful assistant that answers in Korean"
    else:
        system_message = "You are a helpful assistant that answers in English"

    # OpenAI GPT-3.5-turbo 모델을 사용하여 답변 생성
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": f"Here are some documents:\n\n{context}\n\nQuestion: {query}\nAnswer:"}
        ],
        max_tokens=1000  # max_tokens 값을 증가시켜 잘리는 문제 방지
    )
    
    answer = response.choices[0].message['content'].strip()
    return answer, references

In [20]:
# 예시 질문
query = "육아휴직은 애가 몇살까지 쓸 수 있어?"
answer, references = generate_answer(query)

print("답변:", answer)
print("참조한 문서 경로:\n", references)

답변: 문서에는 육아휴직에 대한 정보가 없습니다. 일반적으로는 자녀가 만 8세 이하일 때까지 육아휴직을 사용할 수 있습니다만, 정확한 정보는 해당 근로계약서나 법률에 따라 다를 수 있습니다.
참조한 문서 경로:
 c:\Users\wonhy\OneDrive\Desktop\gist-policy-bot\Data\FR00601 취업규칙_시간외야간및휴일근무.txt
